<a href="https://colab.research.google.com/github/HarunHarmann/Teknofest/blob/main/'Pozisyon%20Kestirimi'/SIFT_Pozisyon_Kestirimi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import cv2
import os
from decimal import Decimal
import numpy as np
from google.colab.patches import cv2_imshow
import pandas as pd

In [ ]:
ground_truth = pd.read_csv("GT_Translations.csv")
ground_truth

## Görüntülerin ve kalibrasyon parametrelerinin değişkene atanması

In [ ]:
# img_path = "/content/drive/MyDrive/Teknofest/Datasets/Pozisyon_Kestirimi/Oturum1_Part2"
img_path = "/content/drive/MyDrive/Teknofest/Datasets/frame/"
file_names = sorted([file_name for file_name in os.listdir(img_path) if file_name.endswith(('.jpg', '.png'))])

# Görüntü dosya yollarını bir listeye ekleyin
image_files = [os.path.join(img_path, file_name) for file_name in file_names]

# Görüntüleri bir diziye ekleyin
images = [cv2.imread(image) for image in image_files]

print(len(images))

In [ ]:
intrinsicMatrix = None
focalLength = None
principalPoint = None
radialDistortion = (-0.0091, 0.0666)
tangentialDistortion = (0,0)
imageSize = None
rotationMatrices = None
translationVectors = None
meanReprojectionError = None
reprojectionErrors = None
reprojectedPoints = None
numPatterns = None
worldPoints = None
worldUnits = None
estimateSkew = None
numRadialDistortionCoefficients = None
estimateTangentialDistortion = None

# /content/drive/MyDrive/Teknofest/Datasets/Pozisyon_Kestirimi/kalibrasyon.txt
with open('kalibrasyon.txt', 'r') as file:
    file_contents = file.readlines()


# Değişkenlere atama
for line in file_contents:
    if 'IntrinsicMatrix' in line:
      intrinsicMatrix = [[0.0 for _ in range(int(line.split(':')[1][4]))] for _ in range(int(line.split(':')[1][2]))]
      print(intrinsicMatrix)
    elif 'FocalLength' in line:
      focalLength = (float(line.split(':')[1].split()[0][1:]),float(line.split(':')[1].split()[1][:-1]))
      print(focalLength)
    elif 'PrincipalPoint' in line:
      principalPoint = (float(line.split(':')[1].split()[0][1:]),float(line.split(':')[1].split()[1][:-1]))
      print(principalPoint)
    # elif 'RadialDistortion' in line:
    #   print(float(line.split(':')[1].split()[1][:-1]))
    #   radialDistortion = (float(line.split(':')[1].split()[0][1:]),float(line.split(':')[1].split(" ")[1][:-1]))
    #   print(radialDistortion)
    # elif 'TangentialDistortion' in line:
    #   tangentialDistortion = (float(line.split(':')[1].split()[0][1:]),float(line.split(':')[1].split()[1][:-1]))
    #   print(tangentialDistortion)
    elif 'ImageSize' in line:
      imageSize = (int(line.split(':')[1].split()[0][1:]),int(line.split(':')[1].split()[1][:-1]))
      print(imageSize)
    elif 'RotationMatrices' in line:
      part = line.split(':')[1].split(" ")[1].split("x")
      rotationMatrices = [[ [0.0 for col in range(int(part[2]))] for col in range(int(part[1]))] for row in range(int(part[0][1:]))]
      print(rotationMatrices)
    elif 'TranslationVectors' in line:
      part = line.split(':')[1].split(" ")[1].split("x")
      translationVectors = [[0.0]*int(part[1])]*int(part[0][1:])
      print(translationVectors)
    elif 'MeanReprojectionError' in line:
      meanReprojectionError = float(line.split(':')[1])
      print(meanReprojectionError)
    elif 'ReprojectionErrors' in line:
      part = line.split(':')[1].split(" ")[1].split("x")
      reprojectionErrors = [[ [0.0 for col in range(int(part[2]))] for col in range(int(part[1]))] for row in range(int(part[0][1:]))]
      print(reprojectionErrors)
    elif 'ReprojectedPoints' in line:
      part = line.split(':')[1].split(" ")[1].split("x")
      reprojectedPoints = [[ [0.0 for col in range(int(part[2]))] for col in range(int(part[1]))] for row in range(int(part[0][1:]))]
      print(reprojectedPoints)
    elif 'NumPatterns' in line:
      numPatterns = int(line.split(':')[1])
      print(numPatterns)
    elif 'WorldPoints' in line:
      part = line.split(':')[1].split(" ")[1].split("x")
      worldPoints = [[0.0]*int(part[1])]*int(part[0][1:])
      print(worldPoints)
    elif 'WorldUnits' in line:
      worldUnits = line.split(':')[1].replace("'"," ").strip()
      print(worldUnits)
    elif 'EstimateSkew' in line:
      estimateSkew = int(line.split(':')[1])
      print(estimateSkew)
    elif 'NumRadialDistortionCoefficients' in line:
      numRadialDistortionCoefficients = int(line.split(':')[1])
      print(numRadialDistortionCoefficients)
    elif 'EstimateTangentialDistortion' in line:
      estimateTangentialDistortion = int(line.split(':')[1])
      print(estimateTangentialDistortion)

##### Intrinsic Matrix değerlerinin verlmesi


In [ ]:
intrinsicMatrix[0][0] = focalLength[0]
intrinsicMatrix[1][1] = focalLength[1]
intrinsicMatrix[0][2] = principalPoint[0]
intrinsicMatrix[1][2] = principalPoint[1]
intrinsicMatrix[2][2] = 1
intrinsicMatrix

### Görüntülerin Ön İşlemesi (Gri Tonlama)

In [ ]:
for i, img in enumerate(images):
    # Görüntüyü gri tona dönüştür
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray_img = cv2.normalize(gray_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    if(i == 1):
      cv2_imshow(gray_img)
    # Gri tona dönüştürülmüş görüntüyü aynı indeksteki görüntü ile güncelle
    images[i] = gray_img

SIFT ile Keypoint Detection

In [ ]:
import cv2
previous_frame = images[0]
current_frame = images[1]
# SIFT kullanarak anahtar noktalarını ve tanımlayıcıları bulma
sift = cv2.SIFT_create()

# Birinci ve ikinci görüntüdeki anahtar noktalarını ve tanımlayıcıları bulun
keypoints1, descriptors1 = sift.detectAndCompute(previous_frame, None)
keypoints2, descriptors2 = sift.detectAndCompute(current_frame, None)

# BFMatcher kullanarak tanımlayıcıları eşleştirin
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)

# Eşleşmeleri sıralayın
matches = sorted(matches, key=lambda x: x.distance)

# Eşleşen noktaları ayıklayın
points1 = np.array([keypoints1[m.queryIdx].pt for m in matches])
points2 = np.array([keypoints2[m.trainIdx].pt for m in matches])

In [ ]:
def to_world_coordinates(points, K):
    points_h = np.hstack((points, np.ones((points.shape[0], 1))))
    world_points = np.linalg.inv(K).dot(points_h.T).T
    return world_points[:, :2]  # X ve Y koordinatlarını alın

world_points1 = to_world_coordinates(points1, intrinsicMatrix)
world_points2 = to_world_coordinates(points2, intrinsicMatrix)

# Hareket vektörlerini hesaplayın
motion_vectors = world_points2 - world_points1

Pozisyon tespitinin yapılması

In [ ]:
from sklearn.linear_model import RANSACRegressor, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
import numpy as np


pixel_size_x = imageSize[0] / focalLength[0]
pixel_size_y = imageSize[1] / focalLength[1]

# Dönüşüm faktörünü hesaplama (conversion factor)
conversion_factor = np.array([1 / pixel_size_x, 1 / pixel_size_y])


# RANSAC modelini ayarla
degree = 2  # Polinomal derecesi
polynomial_regression = make_pipeline(PolynomialFeatures(degree), Ridge())
ransac = RANSACRegressor(polynomial_regression,min_samples=0.8, residual_threshold=1.0, max_trials=1500, stop_score=0.99)

height, width = flow.shape[:2]
X = np.indices((height, width)).reshape(2, -1).T  # Piksel koordinatlarını oluştur
y_x = flow[:, :, 0].flatten()  # X yönündeki hareket vektörleri
# X yönündeki hareket için RANSAC modelini eğit
# ransac.fit(X, y_x)
# y_pred_x = ransac.predict(X) * conversion_factor[0]

# X yönündeki hareket için RANSAC modelini eğit
ransac.fit(world_points1, motion_vectors[:, 0])
y_pred_x = ransac.predict(world_points1)*conversion_factor[0]
# Y yönündeki hareket için RANSAC modelini eğit
X = points1  # Birinci görüntüdeki noktalar
y_y = points2[:, 1] - points1[:, 1]  # Y yönündeki hareket
ransac = RANSACRegressor(min_samples=0.5, residual_threshold=5.0, max_trials=500, stop_score=0.99)
ransac.fit(world_points1, motion_vectors[:, 1])
y_pred_y = ransac.predict(world_points1)*conversion_factor[1]

# y_pred_y = ransac.predict(X) * conversion_factor[1]

# Ortalama hareketi hesapla
avg_movement_x = np.mean(y_pred_x)
avg_movement_y = np.mean(y_pred_y)

print(f'Ortalama hareket X yönünde: {avg_movement_x}')
print(f'Ortalama hareket Y yönünde: {avg_movement_y}')